<a href="https://colab.research.google.com/github/Greta-gerasimov/ALG_2/blob/Lesson_6_GBoosting%2FSGB/stochastic_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

import numpy as np
import xgboost
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# loading data
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
#visualisation

colors = ListedColormap(['red', 'blue'])
light_colors = ListedColormap(['lightcoral', 'lightblue'])


plt.figure(figsize=(8,8))
plt.scatter(list(map(lambda x: x[0], X)), list(map(lambda x: x[1], X)), 
              c = y, cmap=colors)
plt.show()

In [ ]:

# grid search
model = XGBClassifier()
colsample_bylevel = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
param_grid = dict(colsample_bylevel=colsample_bylevel)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
pyplot.errorbar(colsample_bylevel, means, yerr=stds)
pyplot.title("XGBoost colsample_bylevel vs Log Loss")
pyplot.xlabel('colsample_bylevel')
pyplot.ylabel('Log Loss')
pyplot.savefig('colsample_bylevel.png')

In [ ]:
# multiclass classification


def MSE(y_real, predictions):
    return (sum((y_real - predictions)**2)) / len(y_real)



test_size = 0.25
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size, random_state =42)
# fit model no training data
model = xgboost.XGBClassifier(max_depth=5, subsample = 0.2,
                  n_estimators = 100, seed = 42, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
print(model)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#evaluating predictions
rmse = np.sqrt(MSE(y_test, y_pred))
print("RMSE : % f" %(rmse))
model.score(X_test, y_test)


In [ ]:
print('XGBClassifier', classification_report(y_test, y_pred), sep = '\n')

In [ ]:
from xgboost import plot_tree
plot_tree(model)
plt.show()

In [ ]:
from sklearn.metrics import r2_score as r2
import seaborn as sns
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Valid R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    pyplot.figure(figsize=(18,10))
    
    pyplot.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    pyplot.xlabel('Predicted values')
    pyplot.ylabel('True values')
    pyplot.title('Train sample prediction')
    
    pyplot.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    pyplot.xlabel('Predicted values')
    pyplot.ylabel('True values')
    pyplot.title('Valid sample prediction')

    pyplot.show()

In [ ]:
y_train_preds = model.predict(X_train)
y_train_preds = model.predict(X_test)

evaluate_preds(y_train, y_pred_train, y_test, y_pred)

In [ ]:
colors = ListedColormap(['red', 'blue'])
light_colors = ListedColormap(['lightcoral', 'lightblue'])


plt.figure(figsize=(8,8))
plt.scatter(y_test,y_pred, 
              c = y_test, cmap=colors)